In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

# Access the environment variables
# env_vars = os.environ

True

In [5]:
from pprint import pprint
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import Annotated
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage,  SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.graph import MessagesState

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0) 

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))